<a href="https://colab.research.google.com/github/lszam/bibtex/blob/main/bibtex_algoritmocompleto_22_jan_2024_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Formatacao de bibliografias a partir de arquivos bibtex
Programa Python para criar e ler arquivos BibTeX, e usar um arquivo .csl (Citation Style Language) para produzir uma lista de referências bibliográficas. Agradecimentos às recomendações em https://marcel.bollmann.me/blog/turning-bibtex-into-bibliographies-with-python/

Orientações em português para conteúdo do arquivo .bib: chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/http://paginapessoal.utfpr.edu.br/jamhour/publicacoes/arquivos/00_Compilado_JabRef_dez2015.pdf


######Autora: Luizemara Szameitat - Repositorio GitHub https://github.com/lszam/bibtex/

Upload do arquivo de lista de referencias.

In [ ]:
from google.colab import files
import io
#Fazer upload do arquivo .csv
uploaded=files.upload()

Saving lista_de_referencias.csv to lista_de_referencias.csv


#Criar arquivos BIBTEX

In [ ]:
def author_surname_name(author):
  '''
  Formatting author's name as 'LastSurname, Name OtherSurnames'
  '''
  author_split = author.split()
  if ',' in author_split[0]:
    return author
  else:
    if len(author_split) > 1:
      return author_split[-1] + ", " + " ".join(author_split[:-1])
    else:
      return author

# Exemplos de uso
author1 = "Sobrenome1, Nome1 NomeDoMeio1"
# author2 = "Nome2 NomeDoMeio2 Sobrenome2"
author2 = "Maisha Amaru"
formatted_author1 = author_surname_name(author1)
formatted_author2 = author_surname_name(author2)
print(formatted_author1)
print(formatted_author2)

Sobrenome1, Nome1 NomeDoMeio1
Amaru, Maisha


##Função DOI
Tratada a incompatibilidade, converte a entrada para utf-8; tratada a entrada do site Zenodo, espaços entorno do '='.


In [ ]:
import requests
import re
import chardet

def doi2bibfile_quotes(doi):
    error_log = list()
    doi_pages = ['https://doi.org/',
                 'https://dx.doi.org/',
                 'http://doi.org/',
                 'http://dx.doi.org/',
                 'https://zenodo.org/doi/']
    headers = {'Accept': 'application/x-bibtex'}

    for item in doi_pages:
        try:
            url = f'{item}{doi}'
            response = requests.get(url, headers=headers)
            encoding = chardet.detect(response.content)['encoding']
            response_utf8 = response.content.decode('utf-8', 'replace')

            if response.status_code == 200:
                # print(f'{url} : dados obtidos com sucesso')
                break
        except Exception as e:
            print('EXCEPTION: URL', url, str(e))
            error_log.append([doi, url, str(e)])

    if response:
        # Conteúdo como BibTeX
        bibtex_data = response_utf8

        #Type, Nickname
        type_match = re.search(r'@([^{}]+){([^,]+),', bibtex_data)
        entry_type = str(f'{type_match.group(1)}')
        nick = type_match.group(2)

        # Informações essenciais: title, author, year
        try:

          # Title
          title_match = re.search(r'title\s*=\s*{([^}]+)}', bibtex_data)
          title = title_match[1].replace(",", "*")

          # Author
          author_match = re.search(r'author\s*=\s*{([^}]+)}', bibtex_data)
          author_list = author_match.group(1).split('and')
          # Padronizar o nome dos autores como sobrenome, nome
          # pois este é o padrão mais comum em lista de referencias para artigos
          author_checked_list = list()
          for a in author_list:
            author_name = author_surname_name(a)
            author_checked_list.append(a)
          author = "and".join(author_checked_list)

          # # Substituição temporaria para diferenciar fim de linha com vírgula dos autores
          # author = author_match[1].replace(",", "*")

          # Year
          year_match = re.search(r'year\s*=\s*{(\d+)}', bibtex_data)
          year = year_match.group(1)
        except Exception as e:
          print(f'Exception on basic information (title, author, year): {e}')

        # Volume
        volume_match = re.search(r'volume\s*=\s*{([^}]+)}', bibtex_data)
        if volume_match is not None:
          volume = volume_match[1]
        else:
          volume = ""
        # Number
        number_match = re.search(r'number\s*=\s*{([^}]+)}', bibtex_data)
        if number_match is not None:
          number = number_match[1]
        else:
          number = ""
        # Pages
        pages_match = re.search(r'pages\s*=\s*{([^}]+)}', bibtex_data)
        if pages_match is not None:
          pages = pages_match[1]
        else:
          pages = ""
        # Journal
        journal_match = re.search(r'journal={([^}]+)}', bibtex_data)
        if journal_match is not None:
          journal = journal_match[1]
        else:
          journal = ""
        # Publisher
        publisher_match = re.search(r'publisher\s*=\s*{([^}]+)}', bibtex_data)
        if publisher_match is not None:
          publisher = publisher_match[1]
        else:
          publisher = ""
        # Keywords
        keywords_match = re.search(r'keywords\s*=\s*{([^}]+)}', bibtex_data)
        if keywords_match is not None:
          keywords = keywords_match[1]
        else:
          keywords = ""
        # Abstract
        abstract_match = re.search(r'abstract\s*=\s*{([^}]+)}', bibtex_data)
        if abstract_match is not None:
          abstract = abstract_match[1]
        else:
          abstract = ""
        # ISSN
        issn_match = re.search(r'issn\s*=\s*{([^}]+)}', bibtex_data)
        if issn_match is not None:
          issn = issn_match[1]
        else:
          issn = ""
        # URL
        url_match = re.search(r'url\s*=\s*{([^}]+)}', bibtex_data)
        if url_match is not None:
          url = url_match[1]
        else:
          url = ""
        # Copyright
        copyright_match = re.search(r'copyright\s*=\s*{([^}]+)}', bibtex_data)
        if copyright_match is not None:
          copyright = copyright_match[1]
        else:
          copyright = ""

        bib_dict = {"entry_type" : entry_type,
                    "id" : nick,
                    "title" : title,
                    "author" : author,
                    "year" : year,
                    "volume" : volume,
                    "number" : number,
                    "pages" : pages,
                    "journal" : journal,
                    "publisher" : publisher,
                    "keywords" : keywords,
                    "abstract" : abstract,
                    "issn" : issn,
                    "url" : url,
                    "doi" : doi,
                    "copyright" : copyright}


        # Nome do arquivo de saída
        author_part = author_match.group(1).split(',')[0]# primeiro nome
        title_part = title_match.group(1).split()[:2]# duas primeira palavras
        title_part[0] = re.sub(r'[\/:*?",<>|]', '_', title_part[0]) # caracteres invalidos
        title_part[1] = re.sub(r'[\/:*?",<>|]', '_', title_part[1]) # caracteres invalidos
        title_part = ''.join(title_part)
        output_filename = f'{author_part}{year}_{title_part}.bib'

        # Gravar arquivo .bib
        try:
            with open(output_filename, 'w', encoding='utf-8') as bibfile:
              head = f'@{entry_type}'+"{"+ f'{nick}'
              bibfile.write(head + ',\r\n')
              for i, (key, value) in enumerate(bib_dict.items()):
                if i > 1 and value != "":  # Ignora os 2 primeiros itens
                    if "*" in value:
                        value = value.replace('*', ',')
                    new_line = f'{key}="{value}",'
                    bibfile.write(new_line + '\r\n')
              bibfile.write('}')
              print(f'Arquivo .bib salvo para o DOI {doi}')

        except Exception as e:
            print(f'Exception na gravação do arquivo .bib para o DOI {doi}: ', str(e))
            print("SOURCE:", response_utf8)
            error_log.append([doi, str("Exception: "+str(e))])

    else:
        print(f'Response error: Não foi possível obter os dados para o DOI {doi}')
        error_log.append([doi, "Não foi possível obter os dados para o DOI"])

    return (bib_dict, error_log)


##Função ISBN

In [ ]:
import requests

def isbn2bibfile_quotes(isbn):
    # URL da API do Google Books
    url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"

    error_log = list()

    try:
        # Solicitacao para a API do Google
        response = requests.get(url)

        if response.status_code == 200:
          book_data = response.json() # Converte a resposta JSON em dict Python

          if 'items' in book_data:
              book_info = book_data['items'][0]['volumeInfo']
              # Extrair informacoes
              title_match = book_info.get('title', '')
              author_match = book_info.get('authors', '')
              subtitle_match = book_info.get('subtitle', '')
              publisher_match = book_info.get('publisher', '')
              published_date_match = book_info.get('publishedDate', '')
              pageCount_match = book_info.get('pageCount', '')
              keywords_match = book_info.get('keywords', '')

              id = f'{author_match[0].split()[-1]}{published_date_match}book'
              id = re.sub(r'[\/:*?",<>|]-', '_', id) # caracteres invalidos
              entry_type = 'book'

              # Author
              author_list = author_match[0].split('and')
              # Padronizar o nome dos autores como sobrenome, nome
              # pois este é o padrão mais comum em lista de referencias para artigos
              author_checked_list = list()
              for a in author_list:
                author_name = author_surname_name(a)
                author_checked_list.append(author_name)
              author_checked = "and".join(author_checked_list)

              # # Substituição temporaria para diferenciar fim de linha com vírgula dos autores
              # author_checked = author_checked.replace(",", "#")
              # print ('CONTROLE')

              #Infos com tags usadas no padrao bibtex
              bib_dict = {"entry_type" : entry_type,
                          "id" : id,
                          "title" : title_match,
                          "subtitle" : subtitle_match,
                          "author" : author_checked,
                          "year" : published_date_match,
                          "pages" : pageCount_match,
                          "publisher" : publisher_match,
                          "keywords" : keywords_match,
                          "isbn" : isbn}

              try:
                with open(f'{id}.bib', 'w', encoding='utf-8') as bibfile:
                  head = f'@{entry_type}'+"{"+ f'{id},'
                  bibfile.write(head + '\r\n')
                  # print(head + '\r\n')
                  for item in bib_dict:
                    # if "*" in bib_dict[item]:
                    new_line = f'{item}="{bib_dict[item]}",'
                    # print(new_line + '\r\n')
                    bibfile.write(new_line + '\r\n')
                  bibfile.write('}')
                  print(f'Arquivo .bib salvo para o ISBN {isbn}')
              except:
                print(f'*** ERRO: Não foi possível salvar o arquivo .bib para o ISBN {isbn}')


    except Exception as e:
       # return {'error': str(e)}
       print('ERRO:', str(e))
       error_log.append([isbn, str(e)])

    return (bib_dict, error_log)


# Exemplo de uso da função
isbn = '978-90-5744-139-4'  # Substitua pelo ISBN do livro desejado
isbn2bibfile_quotes(isbn)

Arquivo .bib salvo para o ISBN 978-90-5744-139-4


({'entry_type': 'book',
  'id': 'Amaru2007book',
  'title': 'Global travel time tomography with 3-D reference models',
  'subtitle': 'Globale reistijdentomografie met 3-D referentiemodellen',
  'author': 'Amaru, Maisha',
  'year': '2007',
  'pages': 174,
  'publisher': '',
  'keywords': '',
  'isbn': '978-90-5744-139-4'},
 [])

##Ler o arquivo .csv com lista de números DOI e ISBN

In [ ]:
import pandas as pd

def read_doi_csv(csv_file):
  df = pd.read_csv(csv_file)
  doi_list = df['doi_and_isbn'].tolist()
  return doi_list

csv_file = list(uploaded.keys())[0]
doi_list = read_doi_csv(csv_file)
bibtex_list, log_list = list(), list()

for item in doi_list:
  item = str(item)
  try:
    if item.startswith('10.'):
      bib_dict, log = doi2bibfile_quotes(item)
      bibtex_list.append(bib_dict)
    else:
      bib_dict, log = isbn2bibfile_quotes(item)
      bibtex_list.append(bib_dict)
  except Exception as e:
    print(f'*** Exception para o código {item}:{str(e)}')
    log_list.append([item,  str(e)])

Arquivo .bib salvo para o DOI 10.1029/2020JB021183
Arquivo .bib salvo para o DOI 10.1016/j.jsames.2021.103218
Arquivo .bib salvo para o ISBN 978-90-5744-139-4
Arquivo .bib salvo para o DOI 10.1016/j.epsl.2012.09.012
Arquivo .bib salvo para o DOI 10.1016/j.jsames.2022.104062
Arquivo .bib salvo para o DOI 10.1016/j.precamres.2016.09.014
Arquivo .bib salvo para o ISBN 978-0-521-84396-6
Arquivo .bib salvo para o DOI 10.1126/sciadv.1500815
Arquivo .bib salvo para o DOI 10.1007/978-3-319-61667-4_8
Arquivo .bib salvo para o DOI 10.1016/S0166-2635(09)01620-X
Arquivo .bib salvo para o DOI 10.1016/j.lithos.2020.105484
Arquivo .bib salvo para o DOI 10.1016/0264-3707(95)00028-8
Arquivo .bib salvo para o DOI 10.1016/j.jsames.2012.07.006
Arquivo .bib salvo para o DOI 10.1016/j.jvolgeores.2017.11.011
Arquivo .bib salvo para o DOI 10.1016/j.precamres.2021.106394
Arquivo .bib salvo para o DOI 10.1002/2016GC006369
Arquivo .bib salvo para o DOI 10.1016/0012-821X(96)00023-4
*** Exception para o código 978

In [ ]:
#Ordenar os registros em bibtex_list por ordem alfabética e ano
# bibtex_list.sort(key=lambda x: (x['author'].split(',')[0].split()[0], x['year']))

#Salvar arquivos .bib gerados

In [ ]:
# # Salvar um arquivo .zip dos arquivos
import shutil

shutil.make_archive("/content/content_files", 'zip', '/content/')
files.download('content_files.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Criar lista de referencias (saída em HTML)

##Importações e instalações

In [ ]:
!pip install pybtex -q
!pip install citeproc-py -q
!pip install --no-cache-dir --force-reinstall git+https://github.com/sciunto-org/python-bibtexparser@main -q
#instalar estilos
!pip install citeproc-py citeproc-py-styles -q
# !pip install gitpython -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.9/179.9 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 30.5 MB/s eta 0:00:00


In [ ]:
import os
from citeproc.source.bibtex import BibTeX
from citeproc import CitationStylesStyle, CitationStylesBibliography, formatter
from citeproc_styles import get_style_filepath
from citeproc import Citation, CitationItem
from google.colab import files
import io
# import git

###Formatação do texto no estilo da revista



Verifique se a revista possui um modelo .CSL (Citation Style Language) cadastrado no repositorio (https://github.com/citation-style-language/styles) da biblioteca citeproc-py-styles https://pypi.org/project/citeproc-py-styles/ .
Insira o nome na variável abaixo, como cadastrado no repositório.

In [ ]:
# Estilo pretendido - revista
csl_model = "journal-of-geophysical-research-solid-earth"

###Obter a lista de referencias a partir dos arquivos .bib na pasta /content/

In [ ]:
# Abrir arquivos BibTeX da pasta
folder_path = "/content/"
entries = []
biblist = []
for filename in os.listdir(folder_path):
    if filename.endswith('.bib'):
      try:
          file_path = os.path.join(folder_path, filename)
          with open(file_path, 'r', encoding='utf-8') as file:
            bib_src = BibTeX(file)
            biblist.append(bib_src)  # lista de dicionários
      except:
          print("Erro ao ler dados do arquivo.", filename)
biblist

Erro ao ler dados do arquivo. Artemieva2011-07-28book.bib
Erro ao ler dados do arquivo. cordani1984.bib
Erro ao ler dados do arquivo. Kearey2002-04-26book.bib


/usr/local/lib/python3.10/dist-packages/citeproc/source/bibtex/bibtex.py:89: UserWarning: Unsupported BibTeX field 'organization'
  warn("Unsupported BibTeX field '{}'".format(field))
/usr/local/lib/python3.10/dist-packages/citeproc/source/bibtex/bibtex.py:89: UserWarning: Unsupported BibTeX field 'institution'
  warn("Unsupported BibTeX field '{}'".format(field))


[{'de_morisson_valeriano_2016': Reference(de_morisson_valeriano_2016)},
 {'kir_ly_2021': Reference(kir_ly_2021)},
 {'dragone_2017': Reference(dragone_2017)},
 {'eyles_1993': Reference(eyles_1993)},
 {'schellart_2002': Reference(schellart_2002)},
 {'lithgow_bertelloni_2014': Reference(lithgow_bertelloni_2014)},
 {'evain_2015': Reference(evain_2015)},
 {'zalan1990bacia': Reference(zalan1990bacia)},
 {'szatmari_2016': Reference(szatmari_2016)},
 {'arena_2016': Reference(arena_2016)},
 {'faccenna_2010': Reference(faccenna_2010)},
 {'angiboust_2012': Reference(angiboust_2012)},
 {'manatschal_2015': Reference(manatschal_2015)},
 {'moulin_2010': Reference(moulin_2010)},
 {'juli__2008': Reference(juli__2008)},
 {'cordani2010rio': Reference(cordani2010rio)},
 {'pinto_2019': Reference(pinto_2019)},
 {'affonso_2021': Reference(affonso_2021)},
 {'lei_2023': Reference(lei_2023)},
 {'mohriak_2023': Reference(mohriak_2023)},
 {'zalan2011santos': Reference(zalan2011santos)},
 {'licht2015role': Referen

###Formatar e salvar arquivo de saída

In [ ]:
# Abrir CSL do repositorio público
stylepath = get_style_filepath(csl_model)
bib_style = CitationStylesStyle(stylepath, validate=False)

output_text = ''  #acumular as referências em uma string
reference_list = list()

for bib in biblist:
    # print("bib in biblist:", bib)
    for key, value in bib.items():
        library = CitationStylesBibliography(bib_style, bib, formatter.html)
        item = CitationItem(key)
        library.register(Citation([item]))
        try:
          text = ''.join(library.style.render_bibliography([item])[0])
          reference_list.append(text)
          print("Adicionado - " + text)
        except Exception as e:
          print(f'***Exception: Erro ao tratar informações de {item}')

reference_list.sort()

with open('references.html', 'w', encoding='utf-8') as f:
    f.write('<html><head><title>References</title></head><body>')
    for ref in reference_list:
        f.write(f'<div>{ref}</div>')
    f.write('</body></html>')

files.download('references.html')

Adicionado - de Morisson Valeriano, C..  (2016). The Southern Brasília Belt (pp. 189–203). Springer International Publishing. https://doi.org/10.1007/978-3-319-01715-0_10
Adicionado - Király, Á., Funiciello, F., Capitanio, F. A., &amp; Faccenna, C..  (2021). Dynamic interactions between subduction zones. <i>Global and Planetary Change</i>, <i>202</i>, 103501. https://doi.org/10.1016/j.gloplacha.2021.103501
Adicionado - Dragone, G. N., Ussami, N., Gimenez, M. E., Lince Klinger, F. G., &amp; Chaves, C. A. M..  (2017). Western Paraná suture/shear zone and the limits of Rio Apa, Rio Tebicuary and Rio de la Plata cratons from gravity data. <i>Precambrian Research</i>, <i>291</i>, 162–177. https://doi.org/10.1016/j.precamres.2017.01.029
Adicionado - Eyles, N., &amp; Eyles, C. H..  (1993). Glacial geologic confirmation of an intraplate boundary, in the Paraná basin of Brazil. <i>Geology</i>, <i>21</i>(5), 459. https://doi.org/10.1130/0091-7613(1993)021<0459:GGCOAI>2.3.CO;2
Adicionado - Schell

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>